# Model Stacking for Heart Disease Prediction

## Project Overview
This notebook demonstrates an advanced **ensemble stacking** approach for heart disease prediction. We combine multiple base learners (XGBoost, Random Forest, Logistic Regression) with a meta-learner to achieve superior performance.

### Key Concepts:
- **Stacking**: Combining predictions from multiple models using a meta-learner
- **Feature Engineering**: Creating domain-specific cardiovascular features
- **Pipeline Architecture**: Preventing data leakage and ensuring reproducibility
- **Hyperparameter Tuning**: Optimizing all model components

## 1. Import Required Libraries

Import all necessary libraries for data manipulation, modeling, and visualization.

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
import time
import warnings

# Machine Learning - Model Selection and Validation
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold, cross_val_score

# Machine Learning - Models
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

# Machine Learning - Preprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Calculate comprehensive metrics
from sklearn.metrics import matthews_corrcoef, cohen_kappa_score, log_loss

# Machine Learning - Metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, 
    roc_auc_score, roc_curve, precision_recall_curve, auc,
    log_loss, matthews_corrcoef, cohen_kappa_score,
    average_precision_score
)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Settings
warnings.filterwarnings('ignore')
np.random.seed(42)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [ ]:
# Load dataset
data = pd.read_csv('../datasets/heart.csv')
df = data.copy()

# Display basic information
print("Dataset Shape:", data.shape)
print("\nFirst 5 rows:")
print(data.head())

print("\nDataset Info:")
data.info()

print("\nMissing Values:")
print(data.isnull().sum())

print("\nTarget Distribution:")
print(data['HeartDisease'].value_counts())

### Load and Explore Dataset
Load the heart disease dataset and display basic information about its structure, missing values, and target distribution.

In [ ]:
# Visualize data patterns
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
colors = ['green', 'red']

# Target distribution
sns.countplot(x='HeartDisease', data=data, palette=colors, ax=axes[0, 0])
axes[0, 0].set_title('Heart Disease Distribution')

# Exercise Angina
sns.countplot(x='ExerciseAngina', hue='HeartDisease', data=data, palette=colors, ax=axes[0, 1])
axes[0, 1].set_title('Exercise Angina by Heart Disease')

# Resting ECG
sns.countplot(x='RestingECG', hue='HeartDisease', data=data, palette=colors, ax=axes[1, 0])
axes[1, 0].set_title('Resting ECG by Heart Disease')

# Chest Pain Type
sns.countplot(x='ChestPainType', hue='HeartDisease', data=data, palette=colors, ax=axes[1, 1])
axes[1, 1].set_title('Chest Pain Type by Heart Disease')

# Sex distribution
sns.countplot(x='Sex', hue='HeartDisease', data=data, palette=colors, ax=axes[2, 0])
axes[2, 0].set_title('Sex by Heart Disease')

# Correlation heatmap
corr = data.select_dtypes(include=[np.number]).corr()
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', ax=axes[2, 1])
axes[2, 1].set_title('Correlation Heatmap')

plt.tight_layout()
plt.show()

### Exploratory Data Analysis Visualizations
Create visualizations to understand the relationships between features and heart disease.

## 2. Data Loading and Exploratory Analysis

Load the Heart Disease dataset and visualize key patterns including target distribution, feature relationships, and correlations.

In [ ]:
# Define feature types
binary_features = ['Sex', 'ExerciseAngina']
multi_categorical_features = ['ChestPainType', 'RestingECG', 'ST_Slope']
numeric_features = ['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak']

# Create preprocessing pipelines
numerical_pipeline = Pipeline([('scaler', StandardScaler())])
binary_pipeline = Pipeline([('ordinal', OrdinalEncoder())])
categorical_pipeline = Pipeline([('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))])

preprocessor = ColumnTransformer([
    ('num', numerical_pipeline, numeric_features),
    ('bin', binary_pipeline, binary_features),
    ('cat', categorical_pipeline, multi_categorical_features)
])

# Split data BEFORE any preprocessing (prevents leakage)
X = df.drop('HeartDisease', axis=1)
y = df['HeartDisease']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Define base learners
base_learners = [
    ('xgb', XGBClassifier(random_state=42, eval_metric='logloss')),
    ('rf', RandomForestClassifier(random_state=42)),
    ('lr', LogisticRegression(solver='liblinear', random_state=42))
]

# Create stacking classifier with out-of-fold predictions (prevents leakage)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
stacking_clf = StackingClassifier(
    estimators=base_learners,
    final_estimator=LogisticRegression(solver='liblinear', random_state=42),
    passthrough=False,  # No leakage: meta-learner only sees base model predictions
    cv=skf,             # Uses out-of-fold predictions (no leakage)
    n_jobs=-1
)

# Build and train pipeline (preprocessing inside pipeline prevents leakage)
baseline_pipeline = Pipeline([
    ('preprocessor', preprocessor),  # Fits on train, transforms test
    ('classifier', stacking_clf)
])

print("Training baseline model...")
baseline_pipeline.fit(X_train, y_train)  # Only train data used for fitting

# Evaluate
y_pred = baseline_pipeline.predict(X_test)
y_proba = baseline_pipeline.predict_proba(X_test)[:, 1]

print(f"Baseline Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Baseline ROC-AUC: {roc_auc_score(y_test, y_proba):.4f}")

### Build Baseline Stacking Model
Create preprocessing pipelines and train the baseline stacking classifier.

## 4. Advanced Feature Engineering with Overfitting Prevention

Create comprehensive cardiovascular-specific features while implementing regularization techniques to prevent overfitting.

In [ ]:
def engineer_features(X):
    """
    Simple feature engineering for beginners - only 3 features!
    Focus: Basic interpretable features that make medical sense
    No leakage: stateless transformation (no fitting required)
    """
    X = X.copy()  # Avoid modifying original data

    # 1. Heart Rate Reserve - measures cardiac capacity
    # Formula: Maximum theoretical HR - actual max HR achieved
    # Lower values = less cardiac reserve = higher risk
    X['Heart_Rate_Reserve'] = 220 - X['Age'] - X['MaxHR']
    
    # 2. Age-Cholesterol interaction - captures combined risk
    # Older age + high cholesterol = exponentially higher risk
    X['Age_Chol_Product'] = X['Age'] * X['Cholesterol']
    
    # 3. Blood Pressure Risk Score - normalized BP indicator
    # Normal BP ~120, so divide by 120 to get risk ratio
    X['BP_Risk_Score'] = X['RestingBP'] / 120
    
    return X

# Create feature engineering transformer
feature_engineer = FunctionTransformer(engineer_features, validate=False)

# List engineered features (only 3 features!)
engineered_feature_names = [
    'Heart_Rate_Reserve',    # Cardiac capacity
    'Age_Chol_Product',      # Combined age-cholesterol risk
    'BP_Risk_Score'          # Normalized blood pressure
]

# Complete feature list: 6 original + 3 engineered = 9 total
numeric_features_fe = numeric_features + engineered_feature_names

In [ ]:
# Build pipeline with feature engineering and regularization to prevent overfitting
numerical_pipeline_fe = Pipeline([
    ('scaler', StandardScaler())
])

preprocessor_fe = ColumnTransformer([
    ('num', numerical_pipeline_fe, numeric_features_fe),
    ('bin', binary_pipeline, binary_features),
    ('cat', categorical_pipeline, multi_categorical_features)
])

# 1. XGBoost with regularization
xgb_regularized = XGBClassifier(
    random_state=42, 
    eval_metric='logloss',
    reg_alpha=0.1,      # L1 regularization
    reg_lambda=1.0,     # L2 regularization
    max_depth=4,        # Limit tree depth
    min_child_weight=3, # Require more samples per leaf
    subsample=0.8,      # Sample 80% of data
    colsample_bytree=0.8  # Use 80% of features
)

# 2. RandomForest with regularization
rf_regularized = RandomForestClassifier(
    random_state=42,
    max_depth=15,           # Limit tree depth
    min_samples_split=20,   # Require more samples to split
    min_samples_leaf=4,     # Require more samples per leaf
    max_features='sqrt',    # Limit features per tree
    max_samples=0.8         # Bootstrap 80% of data
)

# 3. Logistic Regression with strong regularization
lr_regularized = LogisticRegression(
    solver='liblinear',
    random_state=42,
    C=0.1,              # Strong regularization (inverse)
    penalty='l2',       # L2 regularization
    max_iter=1000
)

# Regularized base learners
base_learners_fe = [
    ('xgb', xgb_regularized),
    ('rf', rf_regularized),
    ('lr', lr_regularized)
]

# Stacking with regularized meta-learner (no leakage)
stacking_clf_fe = StackingClassifier(
    estimators=base_learners_fe,
    final_estimator=LogisticRegression(
        solver='liblinear', 
        random_state=42,
        C=0.5,          # Moderate regularization for meta-learner
        penalty='l2'
    ),
    passthrough=False,   # No leakage: meta-learner only sees base predictions
    cv=skf,              # Out-of-fold predictions prevent leakage
    n_jobs=-1
)

# Complete pipeline (feature engineering inside pipeline prevents leakage)
complete_pipeline = Pipeline([
    ('feature_engineering', feature_engineer),  # Applied after split
    ('preprocessor', preprocessor_fe),          # Fits on train only
    ('classifier', stacking_clf_fe)
])

## 5. Hyperparameter Optimization with Overfitting Prevention

Optimize model parameters using RandomizedSearchCV while maintaining regularization.

In [ ]:
# Simplified parameter grid focused on key regularization parameters
param_grid = {
    # Meta-learner regularization
    'classifier__final_estimator__C': [0.1, 0.5, 1.0],
    
    # XGBoost parameters
    'classifier__xgb__n_estimators': [100, 200],
    'classifier__xgb__max_depth': [3, 4, 5],
    'classifier__xgb__learning_rate': [0.01, 0.05, 0.1],
    'classifier__xgb__reg_alpha': [0, 0.1, 0.5],
    'classifier__xgb__reg_lambda': [1.0, 2.0],
    
    # RandomForest parameters
    'classifier__rf__n_estimators': [100, 200],
    'classifier__rf__max_depth': [10, 15, 20],
    'classifier__rf__min_samples_split': [5, 10],
    'classifier__rf__min_samples_leaf': [2, 4],
    
    # Logistic Regression regularization
    'classifier__lr__C': [0.1, 0.5, 1.0]
}

# Run hyperparameter search (no leakage: CV done only on train set)
random_search = RandomizedSearchCV(
    estimator=complete_pipeline,
    param_distributions=param_grid,
    n_iter=10,  # Balanced number of iterations
    scoring='roc_auc',
    cv=skf,     # Cross-validation on training data only
    verbose=0,  # 0: silent, 1: progress bar, 2: detailed output
    n_jobs=-1,
    random_state=42,
    return_train_score=True
)

time_start = time.time()
random_search.fit(X_train, y_train)  # Test set never seen during tuning
time_end = time.time()
time_elapsed = time_end - time_start

print(f"Hyperparameter tuning completed in {time_elapsed:.2f} seconds")
print(f"Best CV ROC-AUC: {random_search.best_score_:.4f}")


## 6. Model Evaluation and Performance Analysis

Comprehensive evaluation of the optimized model with multiple metrics and visualizations.

In [ ]:
# Get best model and predictions
best_model = random_search.best_estimator_
y_pred_opt = best_model.predict(X_test)
y_proba_opt = best_model.predict_proba(X_test)[:, 1]

# Calculate metrics
accuracy_opt = accuracy_score(y_test, y_pred_opt)
precision_opt = precision_score(y_test, y_pred_opt)
recall_opt = recall_score(y_test, y_pred_opt)
f1_opt = f1_score(y_test, y_pred_opt)
roc_auc_opt = roc_auc_score(y_test, y_proba_opt)
mcc_opt = matthews_corrcoef(y_test, y_pred_opt)
kappa_opt = cohen_kappa_score(y_test, y_pred_opt)
logloss_opt = log_loss(y_test, y_proba_opt)

# Confusion matrix
cm_opt = confusion_matrix(y_test, y_pred_opt)

# PR curve
precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_proba_opt)
pr_auc_opt = auc(recall_vals, precision_vals)

# Display results
print("Optimized Model Evaluation")
print(f"Accuracy:  {accuracy_opt:.4f}")
print(f"Precision: {precision_opt:.4f}")
print(f"Recall:    {recall_opt:.4f}")
print(f"F1-Score:  {f1_opt:.4f}")
print(f"ROC-AUC:   {roc_auc_opt:.4f}")

### Evaluate Optimized Model
Calculate comprehensive performance metrics for the tuned model.

In [ ]:
# Create performance visualizations
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Confusion Matrix
sns.heatmap(cm_opt, annot=True, fmt='d', cmap='Blues', ax=axes[0])
axes[0].set_title('Confusion Matrix', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Actual', fontsize=12)
axes[0].set_xlabel('Predicted', fontsize=12)

# ROC Curve
fpr_opt, tpr_opt, _ = roc_curve(y_test, y_proba_opt)
axes[1].plot(fpr_opt, tpr_opt, label=f'ROC-AUC = {roc_auc_opt:.3f}', linewidth=2.5, color='coral')
axes[1].plot([0, 1], [0, 1], 'k--', alpha=0.3, label='Random')
axes[1].set_xlabel('False Positive Rate', fontsize=12)
axes[1].set_ylabel('True Positive Rate', fontsize=12)
axes[1].set_title('ROC Curve', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### Visualize Model Performance
Create comprehensive visualizations comparing baseline and optimized models.

## 📊 Baseline vs Optimized Comparison
Compare performance between baseline and optimized models.

In [ ]:
# Calculate baseline metrics
y_pred_baseline = baseline_pipeline.predict(X_test)
y_proba_baseline = baseline_pipeline.predict_proba(X_test)[:, 1]

baseline_metrics = {
    'Accuracy': accuracy_score(y_test, y_pred_baseline),
    'Precision': precision_score(y_test, y_pred_baseline),
    'Recall': recall_score(y_test, y_pred_baseline),
    'F1-Score': f1_score(y_test, y_pred_baseline),
    'ROC-AUC': roc_auc_score(y_test, y_proba_baseline),
    'MCC': matthews_corrcoef(y_test, y_pred_baseline),
}

optimized_metrics = {
    'Accuracy': accuracy_opt,
    'Precision': precision_opt,
    'Recall': recall_opt,
    'F1-Score': f1_opt,
    'ROC-AUC': roc_auc_opt,
    'MCC': mcc_opt,
}

# Create simplified comparison visualization
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

# ROC Curves Comparison
fpr_baseline, tpr_baseline, _ = roc_curve(y_test, y_proba_baseline)
fpr_opt, tpr_opt, _ = roc_curve(y_test, y_proba_opt)

ax.plot(fpr_baseline, tpr_baseline, label=f'Baseline (AUC={baseline_metrics["ROC-AUC"]:.3f})', 
        color='skyblue', linewidth=2.5)
ax.plot(fpr_opt, tpr_opt, label=f'Optimized (AUC={optimized_metrics["ROC-AUC"]:.3f})', 
        color='coral', linewidth=2.5)
ax.plot([0, 1], [0, 1], 'k--', label='Random', linewidth=1.5, alpha=0.5)

ax.set_xlabel('False Positive Rate', fontsize=12, fontweight='bold')
ax.set_ylabel('True Positive Rate', fontsize=12, fontweight='bold')
ax.set_title('ROC Curve: Baseline vs Optimized', fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='lower right', fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print comparison summary
print("\n" + "="*60)
print("Model Performance Comparison")
print("="*60)
print(f"\n{'Metric':<15} {'Baseline':>12} {'Optimized':>12} {'Change':>12}")
print("-"*60)

metrics_to_show = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC', 'MCC']
for metric in metrics_to_show:
    base_val = baseline_metrics[metric]
    opt_val = optimized_metrics[metric]
    change = opt_val - base_val
    print(f"{metric:<15} {base_val:>12.4f} {opt_val:>12.4f} {change:>+12.4f}")

print("="*60)
print(f"\n✅ Optimized model ROC-AUC: {optimized_metrics['ROC-AUC']:.4f}")
print("="*60)

The comparison shows the impact of hyperparameter tuning and feature engineering. The visualization includes metric comparisons, ROC curves, precision-recall curves, and percentage improvements for each metric.

## 7. Feature Importance Analysis

Analyze which features contribute most to predictions using meta-learner coefficients.

In [ ]:
# Get feature importances from Random Forest base model
rf_model = best_model.named_steps['classifier'].named_estimators_['rf']
xgb_model = best_model.named_steps['classifier'].named_estimators_['xgb']

# Get feature names after preprocessing
preprocessor_opt = best_model.named_steps['preprocessor']

# Build complete feature names
all_feature_names = []
all_feature_names.extend(numeric_features_fe)  # All numeric features (original + engineered)
all_feature_names.extend(binary_features)

# Get one-hot encoded categorical feature names
onehot_features = preprocessor_opt.named_transformers_['cat'].named_steps['onehot']
cat_feature_names = onehot_features.get_feature_names_out(multi_categorical_features)
all_feature_names.extend(cat_feature_names)

# Get feature importances from both models
rf_importances = rf_model.feature_importances_
xgb_importances = xgb_model.feature_importances_

# Average the importances
avg_importances = (rf_importances + xgb_importances) / 2

# Create dataframe
feature_importance_df = pd.DataFrame({
    'Feature': all_feature_names,
    'RF_Importance': rf_importances,
    'XGB_Importance': xgb_importances,
    'Avg_Importance': avg_importances
}).sort_values('Avg_Importance', ascending=False)

# Visualize top 20 features
top_20 = feature_importance_df.head(20)

fig, ax = plt.subplots(1, 1, figsize=(12, 8))
y_pos = np.arange(len(top_20))

ax.barh(y_pos, top_20['Avg_Importance'], color='steelblue', alpha=0.7)
ax.set_yticks(y_pos)
ax.set_yticklabels(top_20['Feature'], fontsize=10)
ax.invert_yaxis()
ax.set_xlabel('Average Importance (RF + XGBoost)', fontsize=12, fontweight='bold')
ax.set_title('Top 20 Feature Importances', fontsize=14, fontweight='bold', pad=20)
ax.grid(alpha=0.3, axis='x')

# Add value labels
for i, (idx, row) in enumerate(top_20.iterrows()):
    ax.text(row['Avg_Importance'], i, f' {row["Avg_Importance"]:.4f}', 
            va='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

### Feature Importance Visualization
Visualize which features contribute most to the model's predictions.

## 8. Prediction Confidence and Sample Analysis

Analyze model confidence distribution and examine sample predictions.

In [ ]:
# Get confidence scores
if y_proba_opt.ndim == 1:
    confidence_scores = np.abs(y_proba_opt - 0.5) + 0.5
else:
    confidence_scores = np.max(y_proba_opt, axis=1)

# Calculate confidence distribution
very_high = (confidence_scores >= 0.95).sum()
high = ((confidence_scores >= 0.8) & (confidence_scores < 0.95)).sum()
moderate = ((confidence_scores >= 0.6) & (confidence_scores < 0.8)).sum()
low = (confidence_scores < 0.6).sum()

print("Confidence Distribution:")
print(f"Very High (≥95%): {very_high} ({very_high/len(confidence_scores)*100:.1f}%)")
print(f"High (80-95%): {high} ({high/len(confidence_scores)*100:.1f}%)")
print(f"Moderate (60-80%): {moderate} ({moderate/len(confidence_scores)*100:.1f}%)")
print(f"Low (<60%): {low} ({low/len(confidence_scores)*100:.1f}%)")

# Sample predictions
sample_idx = np.random.choice(len(X_test), 6, replace=False)
print("\nSample Predictions:")
for i in sample_idx:
    actual = y_test.iloc[i]
    pred = y_pred_opt[i]
    prob = y_proba_opt[i, 1] if y_proba_opt.ndim > 1 else y_proba_opt[i]
    status = "✓" if pred == actual else "✗"
    print(f"{status} Patient {i}: Pred={pred}, Prob={prob:.1%}, Actual={actual}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Confidence distribution
conf_labels = ['Low', 'Moderate', 'High', 'Very High']
conf_counts = [low, moderate, high, very_high]
axes[0].bar(conf_labels, conf_counts, color=['red', 'orange', 'lightgreen', 'green'], alpha=0.7)
axes[0].set_ylabel('Count')
axes[0].set_title('Confidence Distribution')
axes[0].grid(alpha=0.3, axis='y')

# Confidence by class
conf_class0 = confidence_scores[y_pred_opt == 0]
conf_class1 = confidence_scores[y_pred_opt == 1]
bp = axes[1].boxplot([conf_class0, conf_class1], labels=['No Disease', 'Disease'], patch_artist=True)
bp['boxes'][0].set_facecolor('lightblue')
bp['boxes'][1].set_facecolor('lightcoral')
axes[1].set_ylabel('Confidence Score')
axes[1].set_title('Confidence by Predicted Class')
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

### Confidence Analysis and Sample Predictions
Analyze prediction confidence levels and examine specific sample predictions.